In [1]:
import openai
client = openai.OpenAI(
    base_url="http://llama_server:8080/", # "http://<Your api-server IP>:port"
    api_key = "no_key"
)

In [2]:
# example tools

def calculate(operation: str) -> float:
    print("Calculating...")
    return eval(operation)


def get_planet_mass(planet) -> float:
    print("Getting mass of planet: ", planet)
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [3]:
from customagent import LLMAgent, Tool, Tools, generate_example_from_tool

toolbox = Tools()
toolbox.add_tool(Tool(calculate.__name__, calculate, "Calculate the result of a mathematical operation", examples=["calculate('2 * 5 / 3')"]))
toolbox.add_tool(Tool(get_planet_mass.__name__, get_planet_mass, "Get the mass of a planet in kilograms", examples=["get_planet_mass('Earth')"]))

In [4]:
toolbox.listtools()

['calculate', 'get_planet_mass']

In [5]:
toolsprompt = toolbox.generate_prompt()
print(toolsprompt)

calculate: Calculate the result of a mathematical operation
e.g. calculate('2 * 5 / 3')

get_planet_mass: Get the mass of a planet in kilograms
e.g. get_planet_mass('Earth')


In [6]:
msg = generate_example_from_tool(client, toolsprompt)
print(msg)

APIConnectionError: Connection error.

In [24]:
system_prompt="""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate(4 * 7 / 3)
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass(Earth)
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass('Earth')
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate(5.972e24 * 2)
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

COMPLETION

Now it's your turn:

"""


In [7]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

{tools}

{examples}

COMPLETION

Now it's your turn:

""".strip().format(tools=toolsprompt, examples = msg)

In [8]:
print(system_prompt)

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate: Calculate the result of a mathematical operation
e.g. calculate('2 * 5 / 3')

get_planet_mass: Get the mass of a planet in kilograms
e.g. get_planet_mass('Earth')

Example session:
 Question: What is the mass of Jupiter in kilograms and how much would it weigh on Earth's surface?
Thought: I need to find the mass of Jupiter first and then calculate its weight on Earth's surface.
Action: get_planet_mass('Jupiter')
PAUSE
Observation: 1.898×10e27

Thought: Now that I have the mass of Jupiter, I need to calculate its weight on Earth's surface.
Action: calculate('1.898e27 * 9.8')
PAUSE
Observation: 1.860e30

Thought: I have the final value neede

In [9]:
toolusingLLM = LLMAgent(client=client, model="phi3.1-mini", system_message=system_prompt)

In [10]:
import re

def loop(agent: LLMAgent, tools: Tools, max_iterations:int = 10, query: str = ""):
    next_prompt = query
    toolslist = tools.listtools()
    for i in range(max_iterations):
        if next_prompt:
            # prompt the agent
            output = agent(next_prompt, ["PAUSE", "COMPLETION"])
            print(output)

            # check if the agent is asking for an action or pause
            if "PAUSE" in output:
                print("...checking... pause")
                # use regex to extract the latest action
                match = re.search(r"Action: (\w+)\('\(([^']+)'\)\)'", output)
                if match:
                    print("...checking for choosen tool: ", match.group(1))
                    print("...checking for arg: ", match.group(2))
                    chosen_tool = match.group(1)
                    arg = match.group(2).strip("'")
                
                if chosen_tool in toolslist:
                    # run the action
                    result = tools.execute_tool(chosen_tool, arg)
                    next_prompt = f"Observation: {result}"
                else:
                    next_prompt = f"Observation: No tools found"
                
                print(next_prompt)
            if "COMPLETION" in output:
                break
                

In [11]:
loop(agent= toolusingLLM, tools=toolbox, query="What is the mass of Venus plus the mass of Mars and divide all that by 3?")

 Thought: To answer this question, I need to get the mass of Venus and Mars, add them together, and then divide the sum by 3.

Action: get_planet_mass('Venus')
PAUSE
...checking... pause
...checking for choosen tool:  get_planet_mass
...checking for arg:  'Venus'
Getting mass of planet:  Venus
Observation: 4.867e+24
 Thought: Now that I have the mass of Venus, I need to get the mass of Mars and then add the two masses together.

Action: get_planet_mass('Mars')
PAUSE
...checking... pause
...checking for choosen tool:  get_planet_mass
...checking for arg:  'Mars'
Getting mass of planet:  Mars
Observation: 6.39e+23
 Thought: I have the masses of both Venus and Mars. I now need to add them together and then divide the sum by 3.

Action: calculate('(4.867e24 + 6.39e23) / 3')
PAUSE
...checking... pause
...checking for choosen tool:  calculate
...checking for arg:  '(4.867e24 + 6.39e23
Calculating...


SyntaxError: '(' was never closed (<string>, line 1)